In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import keras
import time
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

Using TensorFlow backend.


In [3]:
#read data from csv file
dat=pd.read_csv('Book3.csv')
print(dat.head())

                    Date  Hour  DryBulb  DewPnt  SYSLoad   NumDate
0  2004-01-01 00:00:00.0     1       37      25    12230  732000.0
1  2004-01-01 00:00:00.0     2       37      25    11534  732000.0
2  2004-01-01 00:00:00.0     3       39      24    11038  732000.0
3  2004-01-01 00:00:00.0     4       38      22    10777  732000.0
4  2004-01-01 00:00:00.0     5       37      20    10764  732000.0


In [4]:
hdata=pd.read_excel('Holidays.xls')
hdata.dtypes

Date       datetime64[ns]
Holiday            object
dtype: object

In [5]:
holidays=hdata.Date
holidays.head()

0   2004-01-01
1   2004-01-19
2   2004-02-16
3   2004-05-31
4   2004-07-05
Name: Date, dtype: datetime64[ns]

In [39]:
dat['Date']=pd.to_datetime(dat['Date'])
dat.dtypes

Date       datetime64[ns]
Hour                int64
DryBulb             int64
DewPnt              int64
SYSLoad             int64
NumDate           float64
dtype: object

In [40]:
dayofweek=dat.Date.dt.weekday
dayofweek.head()

0    3
1    3
2    3
3    3
4    3
Name: Date, dtype: int64

In [81]:
isworkday = np.in1d(dayofweek,[0,1,2,3,4]) & ~np.in1d(dat['Date'],holidays)
isworkday[22]

False

In [82]:
import scipy
import scipy.signal
#By using the lfilter method get the previous 24 hour averrage temperature 
prev24houravg = scipy.signal.lfilter(np.ones(24) / 24, 1, dat['SYSLoad'])
prev24houravg.shape

(52608,)

In [83]:
prevweeksamehour=np.hstack(((np.ones(168)*-1),(dat['SYSLoad'][167:-1])))
prevweeksamehour.shape

(52608,)

In [100]:
prevdaysamehour=np.hstack(((np.ones(24)*-1),(dat['SYSLoad'][23:-1])))
prevdaysamehour.shape

(52608,)

In [214]:
# split a univariate dataset into train/test sets
def split_inputs(data):
    trainInd = dat.Date < dt.datetime(2008, 1, 1) 
    train = data[trainInd,:]
    #get rid of the first 168 rows because one of the columns has null values
    train= train[168:,:]
    
    testInd = dat.Date>= dt.datetime(2009, 1, 1)
    test=data[testInd,:]
    return train, test

In [247]:
X = np.vstack((dat['DryBulb'],dat['DewPnt'],dat['Hour'],dayofweek,isworkday,prevweeksamehour,prevdaysamehour,prev24houravg))
X=np.transpose(X)
X.shape

(52608, 8)

In [248]:
train_x, test_x = split_inputs(X)
train_x.shape

(34896, 8)

In [249]:
# organize the inputs of the network such that each input is 2 dimensions of size n_rowsx8 
def organize_inputs(data, n_rows):
    X = list()
    start = 0
    # organize each input in the set
    for _ in range(len(data)):
        #define the last row of the input
        end = start + n_rows
         # check that we havent reached the end of the dataset
        if end <= len(data):
            X.append(data[start:end, 0:8])


        start += 1    
    return np.array(X)


In [250]:
#define the number of timesteps or rows that will be in each input
n_rows=8
train_x = organize_inputs(train_x,n_rows)
test_x = organize_inputs(test_x, n_rows)

In [251]:
def split_outputs(outputs, n_rows):
    trainInd =dat.Date< dt.datetime(2008, 1, 1)
    trainY=outputs[trainInd]
    trainY=trainY[167+n_rows:]
    trainY=trainY.as_matrix(columns=None)
    trainY = trainY.reshape((trainY.shape[0], 1))
        
    testInd = dat.Date>=dt.datetime(2009, 1, 1)
    testY = outputs[testInd]
    testY= testY[n_rows-1:]
    testY = testY.as_matrix(columns=None)
    testY=testY.reshape((testY.shape[0],1))
    
    return trainY, testY

In [252]:
trainY,testY=split_outputs(dat['SYSLoad'],n_rows)


C:\Users\afzal\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
C:\Users\afzal\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [255]:
n_timesteps, n_features = train_x.shape[1], train_x.shape[2]
# reshape output into [samples, timesteps, features]
#train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(Dense(50,activation='relu'))
model.add(Dense(1))

model.compile(loss='mae', optimizer='adam')


In [256]:
start_time = time.time()
model.fit(train_x, trainY, epochs=55, batch_size=250, verbose=1)
time.time() - start_time

Epoch 1/55
34889/34889 [==============================] - 19s 543us/step - loss: 4493.6986
Epoch 2/55
34889/34889 [==============================] - 4s 109us/step - loss: 2044.0854
Epoch 3/55
34889/34889 [==============================] - 4s 110us/step - loss: 1773.3927
Epoch 4/55
34889/34889 [==============================] - 4s 109us/step - loss: 978.5642
Epoch 5/55
34889/34889 [==============================] - 4s 110us/step - loss: 591.7577
Epoch 6/55
34889/34889 [==============================] - 4s 109us/step - loss: 563.9559
Epoch 7/55
34889/34889 [==============================] - 4s 109us/step - loss: 530.4036
Epoch 8/55
34889/34889 [==============================] - 4s 108us/step - loss: 474.3731
Epoch 9/55
34889/34889 [==============================] - 4s 108us/step - loss: 413.2973 1
Epoch 10/55
34889/34889 [==============================] - 4s 108us/step - loss: 389.0932
Epoch 11/55
34889/34889 [==============================] - 4s 111us/step - loss: 371.9371
Epoch 12/55
3

224.61079621315002

In [158]:
predictions=model.predict(test_x)

In [159]:
testY = testY.reshape((testY.shape[0], 1))
error=testY-predictions
mae=np.mean(np.absolute(error))
mae

261.1444390835666

In [160]:
errorpercent=np.absolute(error)/testY
mape=np.mean(errorpercent)*100
mape

1.7561067644322217

In [257]:
50.24953746795654
(34.066816329956055+
35.61914682388306+
38.2979531288147+
35.85555171966553+
34.98866891860962)/5

35.76562738418579

In [258]:
(63.878042697906494+
63.0877959728241+
64.22768688201904+
65.2805757522583+
64.97749018669128)/5

64.29031829833984

In [259]:
(80.74736428260803+
81.15298914909363+
81.50050735473633+
82.75677824020386+
83.18136167526245)/5

81.86780014038087

In [260]:
(98.21234464645386+
98.87372779846191+
100.77370023727417+
101.09415793418884)/4

99.7384826540947

In [261]:
(120.23980164527893+
118.64051413536072+
119.59518122673035)/3

119.49183233579

In [262]:
(135.87458419799805+
138.94720888137817+
140.13399815559387)/3

138.31859707832336

In [263]:
(157.0763120651245+
154.95724439620972+
157.3949511051178)/3

156.47616918881735

In [264]:
(174.51775908470154+
188.9434506893158+
189.60694074630737)/3

184.35605017344156

In [265]:
(188.72506046295166+
206.4637429714203+
206.5083785057068+
220.65961718559265)/4

205.58919978141785

In [266]:
(224.48561239242554+
228.6032943725586+
227.17084670066833)/3

226.7532511552175

In [267]:
210.9065179824829
213.29870319366455
219.4241783618927

(252.45973348617554+
258.5893256664276+
260.30627703666687)/3

257.11844539642334

In [268]:
(275.1976366043091+
279.3067436218262+
280.68764090538025)/3

278.3973403771718